In [ ]:
%pip install eurostat

In [ ]:
import pandas as pd
import eurostat
df = eurostat.get_data_df("ei_bsco_m")
df = df.rename(columns={"geo\\TIME_PERIOD": "country"})

In [ ]:
from datetime import datetime
df.columns = [datetime.strptime(f.split("-")[0] + "-" + f.split("-")[1] + "-01", "%Y-%m-%d")
                if f.startswith("20") or f.startswith("19") else f for f in df.columns]

In [ ]:
rg = eurostat.get_dic("ei_bsco_m", 'geo')
realnames_g = { k: v for (k, v) in rg }
realnames_g['DE'] = "Germany"
realnames_g['EA19'] = "EURO"
realnames_g['EU27_2020'] = "EU"

In [ ]:
de_cci = df[(df["country"] == "DE") & 
            (df["indic"] == "BS-CSMCI") & 
            (df["s_adj"] == "SA")]
de_cci = de_cci[[c for c in de_cci.columns 
                   if isinstance(c, datetime) and (c.year>=2019)]]
de_cci.T.plot()

In [ ]:
# Länder wählen
cci_mix = df[(df["country"].isin(["DE", "EA19", "SE"])) & 
             (df["indic"] == "BS-CSMCI") & 
             (df["s_adj"] == "SA")]

# nur bestimmte Spalte behalten und transponieren (Zeit nach unten)
cci_mix = cci_mix[[c for c in cci_mix.columns 
                     if (isinstance(c, datetime) and c.year>=2000) or c == "country"]].set_index("country").transpose()

In [ ]:
cci_mix.plot()

In [ ]:
cci = df[(df["indic"] == "BS-CSMCI") & (df["s_adj"] == "SA")]
cci = cci[["country", datetime(2022,11,1)]].set_index("country")
# echte Namen verwenden
cci.index = [realnames_g[i] for i in cci.index]
cci.plot.barh(figsize=(9, 9))

In [ ]:
cci.sort_values(datetime(2022,11,1)).plot.barh(figsize=(9, 9))

In [ ]:
cci.plot.hist(bins=20)

In [ ]:
csmci = df[(df["indic"] == "BS-CSMCI") & (df["s_adj"] == "SA")]
csmci

In [ ]:
columns = [c for c in csmci.columns if isinstance(c, datetime) and (c.year>=2010)]

csmci_unwrapped = pd.melt(csmci[["country"] + columns].dropna(), 
                          id_vars=["country"], var_name="month", value_name="csmci")
csmci_unwrapped

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
csmci_unwrapped["country"] = [realnames_g[i] for i in csmci_unwrapped["country"]]
plt.figure(figsize=(8, 10))
sns.boxplot(y="country", x="csmci", data=csmci_unwrapped, palette="viridis")

In [ ]:
labels = csmci_unwrapped.groupby("country").agg({"csmci": "median"}).sort_values("csmci").index.values
plt.figure(figsize=(8, 10))
sns.boxplot(y="country", x="csmci", data=csmci_unwrapped, order=labels, palette="viridis")

In [ ]:
de = df[(df["s_adj"] == "SA") & (df["country"] == "DE")].set_index("indic")[columns].transpose()
de

In [ ]:
de.plot.scatter(x="BS-SFSH", y="BS-CSMCI")

In [ ]:
sns.jointplot(x=de["BS-SFSH"], y=de["BS-CSMCI"], scatter=False, kind="reg")

In [ ]:
from scipy.stats import linregress
linregress(de["BS-SFSH"], de["BS-CSMCI"])

In [ ]:
%pip install geopandas

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f europe.geo.json || wget https://github.com/heiseacademy/ml-python/raw/main/01-jupyter/europe.geo.json")

In [ ]:
import geopandas
bl_geo = geopandas.read_file("europe.geo.json")
bl_geo.plot(figsize=(10,10))

In [ ]:
hm = df[(df["indic"] == "BS-CSMCI") & (df["s_adj"] == "SA")]
ghm = pd.merge(hm, bl_geo, left_on="country", right_on='iso_a2', how="outer")
ghm

In [ ]:
geopandas.GeoDataFrame(ghm).plot(column=datetime(2021, 1, 1), 
         legend=True, 
         legend_kwds={'orientation': "horizontal"}, 
         missing_kwds={
           "color": "lightgrey",
    },
         figsize=(10,10))